<a href="https://colab.research.google.com/github/GitEmmSt/Medical_Imaging/blob/main/Practical_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Medical Imaging/practical_session_3

/content/drive/MyDrive/Medical Imaging/practical_session_3


#  Medical Imaging 
##  Practical session 4: Image Processing - Part I 
### Image Enhancement and Filtering
### 23rd November 2021
***
**Jakub Ceranka, Sebastian Amador Sanchez, Jef Vandemeulebroucke\
Department of Electronics and Informatics (ETRO)\
Vrije Universitet Brussel, Pleinlaan 2, B-1050 Brussels, Belgium**

<font color=blue>Insert students names and IDs here</font>

## Introduction
For more information on the following concepts see the lecture recordings, course slides and the related study material.

### Purpose
The goal of this exercise session is to obtain an insight in the image enhancement and filtering operations commonly applied in medical image processing.


### BraTS dataset
You will be working with images obtained from the [*Brain Tumor Segmentation (BRATS) Challenge*](http://www.braintumorsegmentation.org), which contains the scans of multiple glioma cases. 
Gliomas are a type of brain tumor that originate in the glial cells that surround the neurons. They are characterized by having various heterogeneous histological sub-regions. Therefore, they have varying intensity profiles. Consequently, to properly visualize them multimodal MRI scans have to be employed, making multimodal segmentation of brain tumors a major challenge in medical image analysis.

<img src="./images/brats.png" alt="drawing" width="800"/>

**(A)** Whole tumor visible in T2-FLAIR **(B)** Tumor core visible in T2 **(C)** Enhancing tumor (blue) and necrotic component (green) visible in T1-Contrast **(D)** Tumor sub-regions.

You DO NOT have to download the dataset, the images that you will use are included in this practical session. These images were artifically corrupted so that you can apply enhancing and denoising techniques:
- T1c image was corrupted with low contrast
- T2 image was corrupted with salt-and-pepper noise
- Flair image was corrupted with MR bias field signal

At the end of this session, it is exepected that you obtain enhanced and noise-free images where you can apply segmentation algorithms (Practical session 5: Image segmentation).

### Instructions
The jupyter notebook should be submitted as the report by teams of two using assignment functionality of Ufora.

Please complete this notebook and upload the following before the deadline **7th December, 2021, at 23:59**:
- the notebook in *.ipynb* format
- the executed notebook in *.html* format (File --> Download As --> HTML)

The report should contain concise answers to the questions (in specified cells), python code and plotted figures.
For this practical session, **we do not** require a separate written report in *.pdf* format.

This is the first session of a two-parts practicum. Therefore, you will have to save the resultant images for the following session. If you do not manage to generate the desired enhanced and noise-free images, these will be provided in the next session.


#### Questions:  [jceranka@etrovub.be](mailto:jceranka@etrovub.be), [samadors@etrovub.be](mailto:samadors@etrovub.be)

### Required modules
Before starting make sure you have installed the following libraries:

- ```SimpleITK``` -> Read and write images
- ```numpy``` -> Operation with arrays
- ```matplotlib``` -> Plot images
- ```skimage``` -> Filtering

# 1. Image Enhancement
## 1.1 The image histogram
The histogram is a representation of how many pixels have a certain intensity in the corresponding image. In image processeing, it facilitates the identification of image acquisition issues, for example:

- **Over and under exposure:** Are intensity values spread out (good) or clustered (bad)?

<img src="./images/hist_exposure.png" alt="drawing" width="800"/>

- **Contrast:** In the image, are there many distinct intensity values (high contrast) or the image uses few intensity values (low contrast)? A "normal" contrast is when intensity values are widely spread and there is a large difference between min and max intensity values. 

<img src="./images/hist_contrast.png" alt="drawing" width="800"/>

- **Dynamic range:** Related to the number of distinct pixels in the image.

<img src="./images/hist_dyn_range.png" alt="drawing" width="800"/>

Unlike previous examples, medical images, can however have a large intensity range, or even floating point intensities. This yields very large histograms and makes the pixel count per intensity impractical. 

<img src="./images/hist_mri.png" alt="drawing" width="600"/>

Therefore, in practice intensities are usually binned, i.e. grouped in a reduced number of bins with similar intensity.

## 1.2 Image enhancement

We shall discuss two ways of contrast improvement: 

1. [Linear contrast mapping](http://homepages.inf.ed.ac.uk/rbf/HIPR2/stretch.htm) or histogram stretching. It involves a linear transformation on the image intensities, such that the transformed intensities cover to the full range.
2. [Histogram equalisation](https://scikit-image.org/docs/dev/auto_examples/color_exposure/plot_equalize.html). In this case, the aim is to obtain a uniform histogram, in which all intensities are equally represented. This can be done by applying a nonlinear transformation on the image intensities. It can be shown that the transform corresponds to the cumulative histogram.

## Exercise 1.1: Linear contrast mapping


- Start by reading the image "BraTS2021_01666_t1ce.mha" from the folder "BraTS2021_01666" with the command [```ReadImage(path_to_image)```](https://simpleitk.readthedocs.io/en/master/IO.html). 
- Visualize the image, first convert it to an array using ```sitk.GetArrayFromImage(image)```. Next, employ [```imshow(array)```](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.imshow.html) to plot the image. 
- Afterwards, use [```hist(image, bins)```](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.hist.html) from ```matplotlib``` with ```bins=64``` to view the histogram. 

Remember that ```SimpleITK``` returns an ITK object, you will have to convert the image to an array before using '```imshow(image)```' and '```hist(image, bins)```'.

- Write a function that performs linear histogram stretching (see course slides). Look at the result and its histogram with 64 bins. Compare with the histogram of the original.

To built the function:
1. Instead of using the minimum and maximum intensity values, start by obtaining the percentiles (P5 and P95) of the image array using [```np.percentile```](https://numpy.org/doc/stable/reference/generated/numpy.percentile.html).
2. [Clip](https://numpy.org/doc/stable/reference/generated/numpy.clip.html) the image array employing the values obtained previously. In other words, clipping will set all values below the P5 to 0, and all values above P95 to 1.
3. Apply the linear stretching transformation to the clipped image using the percentile values as min and max intensities (P5 and P95 respectively).

## Exercise 1.2: Histogram equalization

- Create a function that implements histogram equalization (see course slides) to the original image using [```np.histogram```](https://numpy.org/doc/stable/reference/generated/numpy.histogram.html). Look to the new histogram using 64 bins. 

To built the function:
1. Retrieve the histogram and the respective bin edges employing ```np.histogram```. You will have to apply ```.ravel()``` to the image array to correctly obtain the values.
2. Calculate the center of the bin edges.
3. Determine the cumulative histogram using [```.cumsum()```](https://numpy.org/devdocs/reference/generated/numpy.cumsum.html)
4. Re-scale the cumulative histogram between 0 and 1 by dividing with the max value of the cumulative histogram.
5. Use [```np.interp()```](https://numpy.org/doc/stable/reference/generated/numpy.interp.html) to apply the new distribution to the image. Since ```np.interp()``` is a one-dimensional linear interpolation, flat the original image array using [```flat```](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flat.html).  Additionally, use the center of the bin edges as the new x-coordinates and the re-scaled cumulative histogram as the new y-coordinates.
6. Since the image of point 5 is a 1D-array, reshape it to the original size using [```.reshape(shape)```](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html).


## Report
<font color=blue> 
- Plot a three-by-two image comparison (use ```subplot```) of the image and corresponding histogram for the original image and the ones obtained through the different methods (i.e. linear contrast mapping and histogram equalisation).
- In the linear constrast mapping step we have introduced lower and upper intensity percentiles. What is the reason for that?
- Look at the output results and their histograms. Compare them with the histogram of the original input image. The histogram of the histogram-equalized output image is not perfectly uniform. What is the reason for this? 
-  Save the linear stretched image using ```save_image()``` function (provided below).
</font>

<font color=blue> Your answer here </font>

In [ ]:
# Function to save images
import os

def save_image(array_to_save, name, ground_truth):
    '''
    This functions converts an array to image domain. Additionally, it creates a subfolder "Results", where the 
    images will be stored.
    Inputs:
        - array_to_save: Image in array format that will be save. Format: numpy array
        - name: Name with which the image is saved. Format: string
        - ground_truth: Corresponds to the ORIGINAL image in SimpleITK format. Format: sitk object 
        
    Outputs:
        - Folder named "Results"
        - New image stored in "Results"
    
    '''
    # Create image
    new_image_sitk = sitk.GetImageFromArray(array_to_save)
    
    # Copy information from the ground truth to the new image
    new_image_sitk.CopyInformation(ground_truth)
    
    # Create folder to save the resultant images
    folder_to_save_images = 'Results'
    if not os.path.isdir(folder_to_save_images):
        os.mkdir(folder_to_save_images)
        
    # Write new image
    sitk.WriteImage(new_image_sitk, os.path.join(folder_to_save_images, '{}.mha'.format(name)))

In [ ]:
# your code here

# 2. Image Denoising

The acquisition of an image is always prone to artifacts that may corrupt or degrade its quality. Examples of them are: noise, blurring and distortion. To reduce the effect of these artifacts, multiple image restoration filters have been proposed. These have been used in medical imaging to enhance or suppress certain features of the images. They may be used either to improve the image quality before reviewing it, or as a preprocessing step to improve the result of further image processing steps such as the segmentation.

<img src="./images/denoising.png" alt="drawing" width="500"/>

## 2.1 Noise suppression.

Image noise can often be assumed to be a high frequency signal. Therefore, many noise reduction approaches filter the high frequency components while preserving the low frequency ones, a common example of these is the 2D-Gaussian filter. 

Despite the wide use of low pass filtering, this technique has the side effect of blurring the edges of the image. To avoid it, smoothing filters that preserve the edges, such as the non-linear median filter, have been proposed.

<img src="./images/noise_removal.png" alt="drawing" width="700"/>

## 2.2 Edge enhancement
The goal is to enhance the edge contrast of an image in an attempt to improve its apparent sharpness. The resultant edge-image can be added to the original image to improve the visual quality, or can be employed as input in an image segmentation approach. 

<img src="./images/edge_enhancement.png" alt="drawing" width="700"/>

## Exercise 2.1

To illustrate image filtering, restore an image which has been distorted with "Salt and Pepper" noise.

1. Read the ground truth image 'BraTS2021_01666_t2.mha' and the noisy one 'BraTS2021_01666_SP.mha'.
2. Apply [Gaussian filtering](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter.html) to the noisy image with a standard deviation of 1.
3. Calculate the filtered and the remaining noise.
4. Calculate the root mean squared difference (RMSD) between the obtained filtered image and the ground truth.
5. Create an edge map of the obtained filtered image using the [prewitt function](https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.prewitt).
6. Repeat the process (steps 3-5), for a [median filter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.median_filter.html) using a kernel of size 3 and an average filter using [ndimage.convolve](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve.html#scipy.ndimage.convolve) for a kernel of size 3. For the average filter you will have to create your own filter kernel.

**hint:** ```RMSD = sqrt([mean_squared_error]```(https://scikit-image.org/docs/dev/api/skimage.metrics.html#skimage.metrics.mean_squared_error)(input_image, filtered_image))


## Report
<font color=blue>
    
- Show a three-by-four plot that displays the following for each method: the resultant filtered image, the filtered noise, the noise that remained and the edge map of the filtered image.     
- Provide all three values for the RMSD between filtered image and the ground truth. Comment briefly on the results.
- What is the interpretation of the difference image with the ground truth and the difference image with the original input image?
- Which filter works best in terms of RMSD and why?
- Which filter preserves the edges the best?
- Save the best result using ```save_image()```.
</font>

<font color=blue> Your answer here </font>

In [ ]:
# your code here

## 2.3 Intensity non-uniformity correction

Non-uniform intensity correction is another common task in image denoising. Grayscale inhomogeneities appear in magnetic resonance (MR) images as systematic changes in the local statistical characteristics of tissues. To reduce these intensity effects Homomorphic Unsharp Masking (HUM) is applied as a post-processing tool.

HUM is conceptually straightforward, can be easily implemented and is very fast. It relies on the assumption that if grayscale inhomogeneities are not present in the image, the mean or median in a local window should match the global mean or median of the overall image. This assumption is approximately true when the filter window is large enough to enclose a representative sample of tissues.

For a detailed implementation see paper: [*''Optimized Homomorphic Unsharp Masking for MR Grayscale Inhomogeneity Correction'' by Benjamin H. Brinkmann, Armando Manduca and Richard A. Robb, IEEE, 1998*](https://ieeexplore.ieee.org/document/700729)

HUM requires the computation of:
- The global mean value $\mu$ of the corrupted image
- The local mean values $\mu_{i,j}$ for each pixel considering a neighbourhood
- The HUM corrected/ideal value of a pixel $f_{i,j} = g_{i,j} \cdot \frac{\mu}{\mu_{i,j}}$, where $g_{i,j}$ is the intensity value of the input image (corrupted/observed image).

## Exercise 2.2

Image 'BraTS2021_01666_bias.mha' is a bias corrupted version of 'BraTS2021_01666_flair.mha'. Implement the HUM algorithm in three different ways to compensate for the artifact:

1. Implement the algorithm straightforward. Because of the size of your local window you will not be able to correct pixels close to the image borders. Use a moving window of size equal to 41 to calculate the local mean.
2. Involve pixels at the image borders by prior padding the image and, thus, enlarging the image. Pad the image with zeros using the half of your window size. To pad the image use [```np.pad```](https://numpy.org/doc/stable/reference/generated/numpy.pad.html)
3. Additional to the padding, try to leave out pixels belonging to the background by using a simple global threshold of 10 over the complete image. In other words, in your calculation of the global mean value do not include the pixels below the threshold.

We expect that for each case you create a function which has the following backbone:
- Calculate global mean image intensity
- Get the half value of the window. Make sure it is in 'int' format.
- Create a template with np.zeros that has the same size as the biased image.
- For points 2 and 3: Before creating the template you will have to pad the biased image with zeros using the half size of your window.
- For point 3: Get global mean intensity of the padded biased image applying the threshold.
- Iterate over the biased image using the window you set, apply the HUM equation: $f_{i,j} = g_{i,j} \cdot \frac{\mu}{\mu_{i,j}}$. Store the new pixel in the template image in a correct location.
- For points 2 and 3: You will have to return to the original image size. To do so you can use ```crop``` function.


After the bias field is removed, calculate the [normalized-root-mean-squared-error](https://scikit-image.org/docs/dev/api/skimage.metrics.html#skimage.metrics.normalized_root_mse) (NRMSE) and the [structural similarity index](https://scikit-image.org/docs/dev/api/skimage.metrics.html#skimage.metrics.structural_similarity)  (SSIM) to evaluate the performance of the denoising algorithms.


**Remarks:** 

- Read the non-bias brain image in '```uint8```' format
- Since you will be padding with zeros, use: $\frac{\mu}{\mu_{i,j} + 1}$
- Make sure the resultant images are in '```uint8```' format

## Report:
<font color=blue>
    
- Plot a one-by-four figure showing the image with bias (BraTS2021_01666_bias.mha) and the three corrected images obtained using the different implementations of the HUM algorithm.
- Provide the values for the NRMSE and SSIM between the three corrected images and the ground truth (BraTS2021_01666_flair.mha).
- Which case had a better performance? Why? Save the best result using ```save_image()```.
    
     </font>

<font color=blue> Your answer here </font>

In [ ]:
# your code here